In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
from arch.__future__ import reindexing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, Activation, LSTM
from keras.optimizers import Adam
import keras

In [ ]:
csv_path = "https://raw.githubusercontent.com/curiousily/Deep-Learning-For-Hackers/master/data/3.stock-prediction/BTC-USD.csv"
df = pd.read_csv('ethereum.csv', parse_dates=['Date'])
df = df.sort_values('Date')
scaler = MinMaxScaler()

close_price = df.Close.values.reshape(-1, 1)

scaled_close = scaler.fit_transform(close_price)
scaled_close = scaled_close[~np.isnan(scaled_close)]
scaled_close = scaled_close.reshape(-1, 1)
SEQ_LEN = 100

def to_sequences(data, seq_len):
    d = []

    for index in range(len(data) - seq_len):
        d.append(data[index: index + seq_len])

    return np.array(d)

def preprocess(data_raw, seq_len, train_split):

    data = to_sequences(data_raw, seq_len)

    num_train = int(train_split * data.shape[0])

    X_train = data[:num_train, :-1, :]
    y_train = data[:num_train, -1, :]

    X_test = data[num_train:, :-1, :]
    y_test = data[num_train:, -1, :]

    return X_train, y_train, X_test, y_test


X_train, y_train, X_test, y_test =\
 preprocess(scaled_close, SEQ_LEN, train_split = 0.95)
DROPOUT = 0.2
WINDOW_SIZE = SEQ_LEN - 1
model = keras.Sequential()
model.add(Bidirectional(LSTM(WINDOW_SIZE, return_sequences=True),
  input_shape=(WINDOW_SIZE, X_train.shape[-1])
))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(
  LSTM((WINDOW_SIZE * 2), return_sequences=True)
))
model.add(Dropout(rate=DROPOUT))

model.add(Bidirectional(
  LSTM(WINDOW_SIZE, return_sequences=False)
))

model.add(Dense(units=1))

model.add(Activation('linear'))
BATCH_SIZE = 64

model.compile(
    loss='mean_absolute_error',
    optimizer='adam'
)

history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=BATCH_SIZE,
    shuffle=False,
    validation_split=0.1
)

y_hat = model.predict(X_test)
y_test_inverse = scaler.inverse_transform(y_test)
y_hat_inverse = scaler.inverse_transform(y_hat)

Epoch 1/50
31/31 [==============================] - 76s 2s/step - loss: 0.0350 - val_loss: 0.0681
Epoch 2/50
31/31 [==============================] - 58s 2s/step - loss: 0.0610 - val_loss: 0.1163
Epoch 3/50
31/31 [==============================] - 58s 2s/step - loss: 0.0366 - val_loss: 0.1847
Epoch 4/50
31/31 [==============================] - 59s 2s/step - loss: 0.0387 - val_loss: 0.0713
Epoch 5/50
31/31 [==============================] - 58s 2s/step - loss: 0.0828 - val_loss: 0.1360
Epoch 6/50
31/31 [==============================] - 61s 2s/step - loss: 0.0493 - val_loss: 0.1240
Epoch 7/50
31/31 [==============================] - 59s 2s/step - loss: 0.0379 - val_loss: 0.0824
Epoch 8/50
31/31 [==============================] - 58s 2s/step - loss: 0.0169 - val_loss: 0.0772
Epoch 9/50
31/31 [==============================] - 57s 2s/step - loss: 0.0396 - val_loss: 0.0900
Epoch 10/50
31/31 [==============================] - 59s 2s/step - loss: 0.0483 - val_loss: 0.0702
Epoch 11/50
31/31 [

In [17]:
arr = []
for i in range(0, len(y_hat_inverse)-10):
        arr.append((y_hat_inverse[i:i+10].std()))
arr1 = []
for i in range(0, len(y_test_inverse)-10):
        arr1.append((y_test_inverse[i:i+10].std()))
arr=np.array(arr)
arr1=np.array(arr1)
abs((1 - (arr/arr1))).mean()

0.41426018492851313

In [12]:
print(y_test_inverse)
file = open("file1.txt", "w+")
 
# Saving the array in a text file
content = str(y_test_inverse)
file.write(content)
file.close()

[[2856.01]
 [2781.47]
 [2940.7 ]
 [2747.46]
 [2692.88]
 [2636.03]
 [2519.98]
 [2229.1 ]
 [2342.3 ]
 [2080.79]
 [1955.36]
 [2009.53]
 [2056.3 ]
 [2145.86]
 [2022.93]
 [2091.14]
 [1912.12]
 [2019.34]
 [1959.85]
 [1975.12]
 [2043.19]
 [1972.93]
 [1979.35]
 [1942.91]
 [1793.42]
 [1724.3 ]
 [1792.68]
 [1813.56]
 [1998.46]
 [1941.79]
 [1818.07]
 [1834.64]
 [1775.29]
 [1804.37]
 [1806.71]
 [1859.99]
 [1813.72]
 [1793.  ]
 [1788.06]
 [1663.02]
 [1531.42]
 [1435.46]
 [1206.67]
 [1210.84]
 [1237.92]
 [1068.49]
 [1086.47]
 [ 995.61]
 [1128.39]
 [1128.32]
 [1125.96]
 [1049.02]
 [1144.19]
 [1225.03]
 [1241.76]
 [1198.03]
 [1190.84]
 [1143.97]
 [1099.51]
 [1069.05]
 [1058.94]
 [1066.6 ]
 [1074.08]
 [1151.19]
 [1133.9 ]
 [1186.45]
 [1238.15]
 [1216.33]
 [1216.94]
 [1167.35]
 [1096.43]
 [1038.56]
 [1115.83]
 [1192.75]
 [1231.03]
 [1356.08]
 [1337.85]
 [1584.77]
 [1543.5 ]
 [1520.64]
 [1575.68]
 [1534.85]
 [1549.61]
 [1598.19]
 [1438.52]
 [1450.7 ]
 [1636.01]
 [1726.01]
 [1720.77]
 [1695.97]
 [1680.  ]